In [2]:
import time
# from data import create_dataset
from models import create_model
from util.visualizer import Visualizer
from fid import FID
from dataset.datahandler import get_data_loader
from rangenet.tasks.semantic.modules.segmentator import *
import yaml
import numpy as np
import torch
import tqdm
import os
from util.lidar import LiDAR
from util import *
from collections import defaultdict
import shutil
from util.sampling.fps import downsample_point_clouds
from util.metrics.cov_mmd_1nna import compute_cov_mmd_1nna
from util.metrics.jsd import compute_jsd
from util.metrics.swd import compute_swd
from util.metrics.seg_accuracy import compute_seg_accuracy

import random


os.environ['LD_PRELOAD'] = "/usr/lib/x86_64-linux-gnu/libstdc++.so.6" 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def cycle(iterable):
    while True:
        for x in iterable:
            yield x

def inv_to_xyz(inv, lidar, tol=1e-8):
    inv = tanh_to_sigmoid(inv).clamp_(0, 1)
    xyz = lidar.inv_to_xyz(inv, tol)
    xyz = xyz.flatten(2).transpose(1, 2)  # (B,N,3)
    xyz = downsample_point_clouds(xyz, 512)
    return xyz



class M_parser():
    def __init__(self, cfg_path, data_dir, data_dir_B, load, is_test, batch_size):
        opt_dict = yaml.safe_load(open(cfg_path, 'r'))
        dict_class = make_class_from_dict(opt_dict)
        members = [attr for attr in dir(dict_class) if not callable(getattr(dict_class, attr)) and not attr.startswith("__")]
        for m in members:
            setattr(self, m, getattr(dict_class, m))
        if data_dir != '':
            self.dataset.dataset_A.data_dir = data_dir
        if data_dir_B != '':
            self.dataset.dataset_B.data_dir = data_dir_B
        if batch_size != '':
            self.training.batch_size = int(batch_size)
        if load != '':
            self.training.checkpoints_dir = os.path.sep.join(cfg_path.split(os.sep)[:-2]) 
        self.training.test = is_test
        self.model.isTrain = self.training.isTrain = not self.training.test
        self.training.epoch_decay = self.training.n_epochs//2



def modify_opt_for_fast_test(opt):
    opt.n_epochs = 2
    opt.epoch_decay = opt.n_epochs//2
    opt.display_freq = 1
    opt.print_freq = 1
    opt.save_latest_freq = 1
    opt.max_dataset_size = 10
    opt.batch_size = 2


def check_exp_exists(opt, cfg_args):
    cfg_path = cfg_args.cfg
    opt_t = opt.training
    opt_m = opt.model
    opt_d = opt.dataset.dataset_A
    modality_A = '_'.join(opt_m.modality_A)
    if hasattr(opt_m, 'modality_cond'):
        cond_modality = '_'.join(opt_m.modality_cond)
    out_ch = '_'.join(opt_m.out_ch)
    if cfg_args.load != '':
        # opt_t.name = cfg_path.split(os.sep)[1]
        opt_t.name = cfg_args.load
        """elif cfg_args.fast_test:
        opt_t.name = 'test'"""
    else:
        if 'pix2pix' in opt_m.name:
            opt_t.name = f'pix2pix_modality_A_{modality_A}_out_ch_{out_ch}_L_L1_{opt_m.lambda_L1}' \
                + f'_L_GAN_{opt_m.lambda_LGAN}_L_mask_{opt_m.lambda_mask}_w_{opt_d.img_prop.width}_h_{opt_d.img_prop.height}' \
                    + f'_netG_{opt_m.netG}_netD_{opt_m.netD}_batch_size_{opt_t.batch_size}_finesize_{opt_d.img_prop.finesize}_lr_{opt_t.lr}'
        elif 'cycle_gan' in opt_m.name:
            opt_t.name = f'cycle_gan_modality_A_{modality_A}_out_ch_{out_ch}_lambda_A_{opt_m.lambda_A}_lambda_B_{opt_m.lambda_B}_lambda_idt_{opt_m.lambda_idt}' \
                + f'_w_{opt_d.img_prop.width}_h_{opt_d.img_prop.height}' \
                    + f'_netG_{opt_m.netG}_netD_{opt_m.netD}_batch_size_{opt_t.batch_size}_finesize_{opt_d.img_prop.finesize}_lr_{opt_t.lr}'
        elif 'gc_gan' in opt_m.name:
            opt_t.name = f'gc_gan_modality_A_{modality_A}_out_ch_{out_ch}_lambda_idt_{opt_m.identity}_lambda_AB_{opt_m.lambda_AB}' \
                + f'_lambda_gc_{opt_m.lambda_gc}_lambda_G_{opt_m.lambda_G}_w_{opt_d.img_prop.width}_h_{opt_d.img_prop.height}' \
                    + f'_netG_{opt_m.netG}_netD_{opt_m.netD}_batch_size_{opt_t.batch_size}_finesize_{opt_d.img_prop.finesize}_lr_{opt_t.lr}'
        elif 'cut' in opt_m.name:
            opt_t.name = f'cut_modality_A_{modality_A}_out_ch_{out_ch}_cond_modality_{cond_modality}_lambda_GAN_{opt_m.lambda_GAN}' \
                + f'_lambda_NCE_{opt_m.lambda_NCE}_lambda_NCE_feat_{opt_m.lambda_NCE_feat}_w_{opt_d.img_prop.width}_h_{opt_d.img_prop.height}' \
                    + f'_netG_{opt_m.netG}_netD_{opt_m.netD}_n_layers_D_{opt_m.n_layers_D}_batch_size_{opt_t.batch_size}_finesize_{opt_d.img_prop.finesize}_lr_decay_iters_{opt_t.lr_decay_iters}_lr_{opt_t.lr}'
        
    exp_dir = os.path.join(opt_t.checkpoints_dir, opt_t.name)
    if not opt_t.continue_train and opt_t.isTrain:
        if os.path.exists(exp_dir):
            reply = ''
            # raise Exception('Checkpoint exists!!')
            while not reply.startswith('y') and not reply.startswith('n'):
                reply = str(input(f'exp_dir {exp_dir} exists. Do you want to delete it? (y/n): \n')).lower().strip()
            if reply.startswith('y'):
                shutil.rmtree(exp_dir)
            else:
                print('Please Re-run the program with \"continue train\" enabled')
                exit(0)
        os.makedirs(exp_dir, exist_ok=True)
        shutil.copy(cfg_path, exp_dir)
    else:
        assert os.path.exists(exp_dir)

params = {
'cfg': 'configs/cut.yaml',  # Path of the config file
'batch_size': '1',  # Batch size
'data_dir': 'Semantic-CARLA/CARLA_Lidar/projected',  # Path of the dataset A
'data_dir_B': '/media/pierre/Main/SemanticKitti/dataset',  # Path of the dataset B
'seg_cfg_path': '',  # Path of segmentator cfg
'load': '',  # Experiment folder name for loading
'test': False,  # Test the model
'map_label': False,  # Map predicted labels in case of Semposs dataset
'norm_label': False,  # Normalize labels
# 'fast_test': False,  # Fast test of the experiment (commented out)
'ref_dataset_name': 'kitti',  # Reference dataset name for measuring unsupervised metrics
'n_fid': 1000,  # Number of samples for FID calculation
'gpu': 0,  # GPU number
'on_input': False,  # Calculate unsupervised metrics on dataset A
'on_real': False,  # Input is real data
'no_inv': False  # Use it for unsupervised metrics on input inv
}

# Simulating argparse arguments
class Args:
    def __init__(self, params):
        for key, value in params.items():
            setattr(self, key, value)

# Use the simulated arguments
cl_args = Args(params)


if 'checkpoints' in cl_args.cfg:
    cl_args.load = cl_args.cfg.split(os.path.sep)[-2]
opt = M_parser(cl_args.cfg, cl_args.data_dir, cl_args.data_dir_B, cl_args.load, cl_args.test, cl_args.batch_size)
if cl_args.on_real:
    opt.dataset.dataset_A.name = cl_args.ref_dataset_name

opt.model.norm_label = cl_args.norm_label
torch.manual_seed(opt.training.seed)
np.random.seed(opt.training.seed)
random.seed(opt.training.seed)
    
# DATA = yaml.safe_load(open(pa.cfg_dataset, 'r'))
## test whole code fast

if not opt.training.isTrain:
    opt.training.n_epochs = 1

check_exp_exists(opt, cl_args)

is_two_dataset = False
if hasattr(opt.dataset, 'dataset_B'):
    is_two_dataset = True
print("is_two_dataset", is_two_dataset)
opt.training.gpu_ids = [cl_args.gpu]
device = "cpu"
ds_cfg = make_class_from_dict(yaml.safe_load(open(f'configs/dataset_cfg/{opt.dataset.dataset_A.name}_cfg.yml', 'r')))
if not hasattr(opt.dataset.dataset_A, 'data_dir'):
    opt.dataset.dataset_A.data_dir = ds_cfg.data_dir
if is_two_dataset:
    print(hasattr(opt.dataset.dataset_B, 'data_dir'))
    ds_cfg_B = make_class_from_dict(yaml.safe_load(open(f'configs/dataset_cfg/{opt.dataset.dataset_B.name}_cfg.yml', 'r')))
    if not hasattr(opt.dataset.dataset_B, 'data_dir'):
        opt.dataset.dataset_B.data_dir = ds_cfg_B.data_dir
ds_cfg_ref = make_class_from_dict(yaml.safe_load(open(f'configs/dataset_cfg/{cl_args.ref_dataset_name}_cfg.yml', 'r')))
lidar_A = LiDAR(
cfg=ds_cfg,
height=opt.dataset.dataset_A.img_prop.height,
width=opt.dataset.dataset_A.img_prop.width).to(device)
lidar_B = LiDAR(
cfg=ds_cfg_B,
height=opt.dataset.dataset_B.img_prop.height,
width=opt.dataset.dataset_B.img_prop.width,
).to(device) if is_two_dataset else None
lidar_ref = LiDAR(
cfg=ds_cfg_ref,
height=opt.dataset.dataset_A.img_prop.height,
width=opt.dataset.dataset_A.img_prop.width).to(device)
lidar = lidar_B if is_two_dataset else lidar_ref
visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
g_steps = 0
min_fid = 10000
if cl_args.ref_dataset_name == 'kitti':
    ignore_label = [0, 2, 3, 4, 5, 7, 8, 10, 12, 16]
elif cl_args.ref_dataset_name == 'semanticPOSS':
    ignore_label = [0, 3, 9]

is_ref_semposs = cl_args.ref_dataset_name == 'semanticPOSS'
train_dl, train_dataset = get_data_loader(opt, 'train', opt.training.batch_size,is_ref_semposs=is_ref_semposs)
val_dl, val_dataset = get_data_loader(opt, 'val' if (opt.training.isTrain or cl_args.on_input)  else 'test', opt.training.batch_size, shuffle=False, is_ref_semposs=is_ref_semposs)  
test_dl, test_dataset = get_data_loader(opt, 'test', opt.training.batch_size, dataset_name=cl_args.ref_dataset_name, two_dataset_enabled=False, is_ref_semposs=is_ref_semposs)
with torch.no_grad():
    seg_model = Segmentator(dataset_name=cl_args.ref_dataset_name if cl_args.seg_cfg_path == '' else 'synth', cfg_path=cl_args.seg_cfg_path).to(device)
model = create_model(opt, lidar_A, lidar_B=lidar_B)      # create a model given opt.model and other options
model.set_seg_model(seg_model)               # regular setup: load and print networks; create schedulers
## initilisation of the model for netF in cut
train_dl_iter = iter(train_dl)
data = next(train_dl_iter)
model.data_dependent_initialize(data)
model.setup(opt.training)
#fid_cls = FID(seg_model, train_dataset, cl_args.ref_dataset_name, lidar_A) if cl_args.ref_dataset_name!= '' else None
n_test_batch =  len(test_dl)
test_dl_iter = iter(test_dl)
data_dict = defaultdict(list)
N = min(len(test_dataset), len(val_dataset), 1000)

for i in tqdm.tqdm(range(0, N, opt.training.batch_size)):
    data = next(test_dl_iter)
    data = fetch_reals(data, lidar_ref, device, opt.model.norm_label)
    data_dict['real-2d'].append(data['inv'])
    data_dict['real-3d'].append(inv_to_xyz(data['inv'], lidar_ref))
epoch_tq = tqdm.tqdm(total=opt.training.n_epochs, desc='Epoch', position=1)
start_from_epoch = model.schedulers[0].last_epoch if opt.training.continue_train else 0 
    
#### Train & V
# alidation Loop
for epoch in range(start_from_epoch, opt.training.n_epochs):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
    epoch_start_time = time.time()  # timer for entire epoch
    e_steps = 0                  # the number of training iterations in current epoch, reset to 0 every epoch
    # Train loop
    model.train(True)
    train_dl_iter = iter(train_dl)
    n_train_batch = len(train_dl)
    for i in tqdm.tqdm(range(n_train_batch)):  # inner loop within one epoch
        data = next(train_dl_iter)
        g_steps += 1
        e_steps += 1
        model.set_input(data)         # unpack data from dataset and apply preprocessing
        model.optimize_parameters()   # calculate loss functions, get gradients, update network weights
        if False:   # display images on visdom and save images to a HTML file
            current_visuals = model.get_current_visuals()
            if is_two_dataset:
                visualizer.display_current_results('train',current_visuals, g_steps,ds_cfg, opt.dataset.dataset_A.name, lidar_A, ds_cfg_B,\
                        opt.dataset.dataset_B.name,lidar_B)
            else:
                visualizer.display_current_results('train',current_visuals, g_steps,ds_cfg, opt.dataset.dataset_A.name, lidar_A)
        if False:    # print training losses and save logging information to the disk
            losses = model.get_current_losses()
            visualizer.plot_current_losses('train', epoch, losses, g_steps)
        if g_steps % 1000 == 0:
            print('saving the latest model (epoch %d, total_iters %d)' % (epoch, g_steps))
            model.save_networks('latest')
    val_dl_iter = iter(val_dl)
    n_val_batch = 2 if cl_args.fast_test else  len(val_dl)
    ##### validation
    val_losses = defaultdict(list)
    model.train(False)
    tag = 'val' if opt.training.isTrain else 'test'
    val_tq = tqdm.tqdm(total=n_val_batch, desc='val_Iter', position=5)
    dis_batch_ind = np.random.randint(0, n_val_batch)
    data_dict['synth-2d'] = [] 
    data_dict['synth-3d'] = []
    fid_samples = [] if fid_cls is not None else None
    iou_list = []
    m_acc_list = []
    rec_list = [] 
    prec_list = []
    label_map = ds_cfg_ref.kitti_to_POSS_map if is_ref_semposs and cl_args.map_label else None
    for i in range(n_val_batch):
        data = next(val_dl_iter)
        model.set_input(data)
        with torch.no_grad():
            model.calc_supervised_metrics(cl_args.no_inv, lidar_A, lidar)
        
        fetched_data = fetch_reals(data['A'] if is_two_dataset else data, lidar_A, device)
        if cl_args.on_input:
            # assert is_two_dataset == False
            if 'inv' in fetched_data:
                synth_inv = fetched_data['inv']
            if 'reflectance' in fetched_data:
                synth_reflectance = fetched_data['reflectance']
            if 'mask' in fetched_data:
                synth_mask = fetched_data['mask']
        else:
            if hasattr(model, 'synth_reflectance'):
                synth_reflectance = model.synth_reflectance 
            if hasattr(model, 'synth_mask'):
                synth_mask = model.synth_mask
            if hasattr(model, 'synth_inv') and not cl_args.no_inv:
                synth_inv = model.synth_inv
            else:
                synth_inv = fetched_data['inv'] * synth_mask
        
        data_dict['synth-2d'].append(synth_inv)
        data_dict['synth-3d'].append(inv_to_xyz(synth_inv, lidar))

        if fid_cls is not None and len(fid_samples) < cl_args.n_fid:
            synth_depth = lidar.revert_depth(tanh_to_sigmoid(synth_inv), norm=False)
            synth_points = lidar.inv_to_xyz(tanh_to_sigmoid(synth_inv)) * lidar.max_depth
            synth_reflectance = tanh_to_sigmoid(synth_reflectance)
            synth_data = torch.cat([synth_depth, synth_points, synth_reflectance, synth_mask], dim=1)
            fid_samples.append(synth_data)
            if not opt.training.isTrain:
                iou, m_acc, prec, rec = compute_seg_accuracy(seg_model, synth_data * fetched_data['mask'], fetched_data['lwo'], ignore=ignore_label,\
                                                                label_map=label_map)
                iou_list.append(iou.cpu().numpy())
                m_acc_list.append(m_acc.cpu().numpy())
                prec_list.append(prec.cpu().numpy())
                rec_list.append(rec.cpu().numpy())

        if i == dis_batch_ind:
            current_visuals = model.get_current_visuals()
            if is_two_dataset:
                visualizer.display_current_results(tag, current_visuals, g_steps, ds_cfg, opt.dataset.dataset_A.name, lidar_A, ds_cfg_B,\
                        opt.dataset.dataset_B.name, lidar_B)
            else:
                visualizer.display_current_results(tag, current_visuals, g_steps, ds_cfg, opt.dataset.dataset_A.name, lidar_A)

        for k ,v in model.get_current_losses(is_eval=True).items():
            val_losses[k].append(v)
        val_tq.update(1)
    if not opt.training.isTrain:
        avg_m_acc = np.array(m_acc_list).mean()
        iou_avg = np.array(iou_list).mean(axis=0)
        prec_avg = np.array(prec_list).mean(axis=0)
        rec_avg = np.array(rec_list).mean(axis=0)
        label_names = seg_model.learning_class_to_label_name(np.arange(len(iou_avg)), ds_cfg_ref)
        print('avg seg acc:', np.round(avg_m_acc, 2))
        print('iou avg:')
        print_str = ''
        cross_class_iou_avg = iou_avg[iou_avg != 0.0].mean()
        for i, (l, iou) in enumerate(zip(label_names, iou_avg)):
            if iou > 0.0:
                # print_str = print_str + f'{l}:{np.round(iou, 4)} precision:{np.round(prec_avg[i],4)}, recall:{np.round(rec_avg[i],4)} '
                print_str = print_str + f'{l}:{np.round(iou, 4)} '
        print(print_str)
        print('cross-class iou:', np.round(cross_class_iou_avg, 2))
    losses = {k: float(np.array(v).mean()) for k , v in val_losses.items()}
    visualizer.plot_current_losses(tag, epoch, losses, g_steps)
    visualizer.print_current_losses(tag, epoch, e_steps, losses, val_tq)
    
    ##### calculating unsupervised metrics


    for k ,v in data_dict.items():
        if isinstance(v, list):
            data_dict[k] = torch.cat(v, dim=0)[: N]
    scores = {}
    scores.update(compute_swd(data_dict["synth-2d"], data_dict["real-2d"]))
    scores["jsd"] = compute_jsd(data_dict["synth-3d"] / 2.0, data_dict["real-3d"] / 2.0)
    scores.update(compute_cov_mmd_1nna(data_dict["synth-3d"], data_dict["real-3d"], 512, ("cd",)))
    if fid_cls is not None:
        scores['fid'] = fid_cls.fid_score(torch.cat(fid_samples, dim=0))
    if scores["fid"] < min_fid and opt.training.isTrain:
        min_fid = scores["fid"]
        model.save_networks('best')
    visualizer.plot_current_losses('unsupervised_metrics', epoch, scores, g_steps)
    visualizer.print_current_losses('unsupervised_metrics', epoch, e_steps, scores, val_tq)
    if opt.training.isTrain:
        model.update_learning_rate()    # update learning rates in the beginning of every epoch.
    epoch_tq.update(1)
    print('End of epoch %d \t Time Taken: %d sec' % (epoch, time.time() - epoch_start_time))    


is_two_dataset True
True
Subsets  [ 0  1  2  3  4  5  6  7  9 10]
taille data 33992
Subsets  [ 0  1  2  3  4  5  6  7  9 10]
taille data 19130
Subsets  [8]
taille data 0
Subsets  [8]
taille data 4071
Subsets  [11 12 13 14 15 16 17 18 19 20 21]
taille data 20351
Using DarknetNet53 Backbone
Depth of backbone input =  5
Original OS:  32
New OS:  32
Strides:  [2, 2, 2, 2, 2]
Decoder original OS:  32
Decoder new OS:  32
Decoder strides:  [2, 2, 2, 2, 2]
Total number of parameters:  50377364
Total number of parameters requires_grad:  50377364
Param encoder  40585504
Param decoder  9786080
Param head  5780
Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
<util.make_class_from_dict.<locals>.dict_class object at 0x7e9398034340>
initialize network with xavier
initialize network with xavier
initialize network with xavier
initialize network with xavier
model [CUTModel] was created
initialize network with xavier


/home/pierre/Documents/git_folders/CoLiGen/models/networks.py:611: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  patch_id = torch.tensor(patch_id, dtype=torch.long, device=feat.device)


---------- Networks initialized -------------
ResnetGenerator(
  (gumbel): GumbelSigmoid(hard=True, eps=1e-10)
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(2, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Downsample(
      (pad): ReflectionPad2d((1, 1, 1, 1))
    )
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): ReLU(inplace=True)
    (11): Downsample(
      (pad): ReflectionPad2d((1, 1, 1, 1))
    )
    (12): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        

  0%|          | 99/33992 [10:30<59:57:47,  6.37s/it]


RuntimeError: Device index must not be negative